In [ ]:
from classifier import classifier
import pandas as pd
from math import log
from sklearn.metrics import accuracy_score

In [ ]:
class decision_tree(classifier):

    def __init__(self, criterion = 'entropy'):
        self.criterion = criterion

    def gini(self, Y):
        size = len(Y)
        counts = dict()
        for y in Y:
            if y not in counts:
                counts[y] = 0.
            counts[y] += 1.
        gini = 0.
        for key in counts:
            prob = counts[key] / size
            gini += prob * (1-prob)
        return gini


    def entropy(self, Y):
        from math import log

        size = len(Y)
        counts = dict()
        for y in Y:
            if y not in counts:
                counts[y] = 0.
            counts[y] += 1.
        entropy = 0.
        for key in counts:
            prob = counts[key] / size
            entropy -= prob * log(prob,2)
        return entropy


    def split_data(self, X, Y, axis, value):
        return_x = []
        return_y = []

        for x, y in (zip(X, Y)):
            if x[axis] == value:
                reduced_x = x[:axis]
                reduced_x.extend(x[axis+1:])
                return_x.append(reduced_x)
                return_y.append(y)
        return return_x, return_y


    def choose_feature(self, X, Y):
        if self.criterion == 'entropy':
#             print("Entropy")
            entropy = self.entropy(Y)
            best_information_gain = 0.
            best_feature = -1
            for i in range(len(X[0])):  # For each feature
                feature_list = [x[i] for x in X]
                values = set(feature_list)
                entropy_i = 0.
                for value in values:
                    sub_x, sub_y = self.split_data(X, Y, i, value)
                    prob = len(sub_x) / float(len(X))
                    entropy_i += prob * self.entropy(sub_y)
                info_gain = entropy - entropy_i
                if info_gain > best_information_gain:
                    best_information_gain = info_gain
                    best_feature = i
            return best_feature
        else:
#             print("Gini")
            entropy = self.gini(Y)
            best_information_gain = 0.
            best_feature = -1
            for i in range(len(X[0])):  # For each feature
                feature_list = [x[i] for x in X]
                values = set(feature_list)
                entropy_i = 0.
                for value in values:
                    sub_x, sub_y = self.split_data(X, Y, i, value)
                    prob = len(sub_x) / float(len(X))
                    entropy_i += prob * self.gini(sub_y)
                info_gain = entropy - entropy_i
                if info_gain > best_information_gain:
                    best_information_gain = info_gain
                    best_feature = i
            return best_feature


    def class_dict(self, Y):
        classes = dict()
        for y in Y:
            if y not in classes:
                classes[y] = 0
            classes[y] += 1
        return classes


    def majority(self, Y):
        from operator import itemgetter
        # Use this function if a leaf cannot be split further and
        # ... the node is not pure

        classcount = self.class_dict(Y)
        sorted_classcount = sorted(classcount.iteritems(), key=itemgetter(1), reverse=True)
        return sorted_classcount[0][0]


    def build_tree(self, X, Y):
        # IF there's only one instance or one class, don't continue to split
        if len(Y) <= 1 or len(self.class_dict(Y)) == 1:
            return Y[0]

        if len(X[0]) == 1:
            return self.majority(Y)   # TODO: Fix this

        best_feature = self.choose_feature(X, Y)
        if best_feature < 0 or best_feature >= len(X[0]):
            return

        this_tree = dict()
        feature_values = [example[best_feature] for example in X]
        unique_values = set(feature_values)
        for value in unique_values:
            # Build a node with each unique value:
            subtree_x, subtree_y = self.split_data(X, Y, best_feature, value)
            if best_feature not in this_tree:
                this_tree[best_feature] = dict()
            if value not in this_tree[best_feature]:
                this_tree[best_feature][value] = 0
            this_tree[best_feature][value] = self.build_tree(subtree_x, subtree_y)
        return this_tree


    def fit(self, X, Y):
        self.fittedTree = self.build_tree(X, Y)
        return self.fittedTree

    def predict(self, X):
        lista = []
        for i in range(len(X)):
            val = self.recursivecall(X[i], self.fittedTree)
            lista.append(val)
        return lista
        
    def recursivecall(self, X, tree):
        if isinstance(tree,int):
            return tree
        keys = tree.keys()
        for k in keys:
            a = X[k]
            if isinstance(tree[k],dict):
                newtree = tree[k][a]
                return self.recursivecall(X, newtree)
            else:
                return tree[k]

In [3]:
entr = decision_tree()
# entr = decision_tree(criterion='gini')

In [4]:
df = pd.read_csv("zoo.csv")
df.head(5)

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [5]:
X = df.iloc[:,1:17]
Y = df.iloc[:,-1]

In [6]:
# split = int(len(X) * 0.86)
# train_data = X[:split]
# train_labels = Y[:split]
# test_data = X[split:]
# test_labels = Y[split:]

split = int(len(X) * 0.20)
train_data = X[split:]
train_labels = Y[split:]
test_data = X[:split]
test_labels = Y[:split]

# test_data = test_data.drop(test_data.index[5])

train_data = train_data.values.tolist()
train_labels = train_labels.values.tolist()

test_data = test_data.values.tolist()
test_labels = test_labels.values.tolist()

tree  = entr.fit(train_data, train_labels)

print('Tree :',tree)

hyp = entr.predict(test_data)

print('Accuracy : ', accuracy_score(test_labels,hyp))

Tree : {12: {0: {11: {0: {7: {0: 7, 1: 3}}, 1: {2: {0: 1, 1: 4}}}}, 2: {0: {0: 2, 1: 1}}, 4: {0: {0: {4: {0: 3, 1: 5}}, 1: 1}}, 5: 7, 6: {5: {0: 6, 1: 7}}, 8: 7}}
Accuracy :  0.95


In [7]:
entr.choose_feature(train_data, train_labels)

12